In [ ]:
import sys
print("--sys.version—")
print(sys.version)

In [ ]:
# import mysql.connector
# db = mysql.connector.connect(host = "14.35.220.97",
#                   user = 'seominseok',
#                   password = 'pintech801',
#                   database = 'dbstock',
#                   charset = 'utf8')
# cursor=db.cursor()

In [1]:
%%time
import pandas as pd
import numpy as np
import sys
from PyQt5.QtWidgets import *
from PyQt5.QAxContainer import *
from PyQt5.QtCore import *
import datetime
import time
import MySQLdb
import mysql.connector
import os

# 적재할 DB
# db = mysql.connector.connect(host = "14.35.220.97",
#                              user = 'seominseok',
#                              password = 'pintech801',
#                              database = 'dbstock',
#                              charset = 'utf8'
#                              local_infile=1)
# cursor=db.cursor()

db = MySQLdb.connect(host = '127.0.0.1', user = 'root', password='1234', charset='utf8', local_infile=1)
cursor = db.cursor()
cursor.execute('use stock_db;')

kospi_df = pd.read_csv('final_code.csv',index_col=0, dtype={'code':str})
now = datetime.datetime.now().strftime('%Y-%m-%d')
day = now.split('-')[2]
today = ''.join(now.split('-'))

time_set ='2021-12-13 09:00:00'
timestp_now = pd.to_datetime(time_set)
time_list=[]
day = timestp_now + datetime.timedelta(days=0)
time_list.append(day)
for m in range(1, 391):
    minutes = day + datetime.timedelta(minutes=0 + m)
    time_list.append(minutes)
df_1month_time = pd.DataFrame({'체결시간' : time_list})
df_1month_time = df_1month_time.sort_values(by='체결시간', ascending=False)
df_1month_time = df_1month_time.set_index('체결시간')
df_1month_time = pd.Series(df_1month_time.index)
base_df = pd.DataFrame({'체결시간':list(df_1month_time.apply(lambda x:str(x)))})
base_df

os.chdir('C:/csv_data')
path = os.getcwd()+'/stock_csv/{}/'.format(today)
if not os.path.isdir(path):
    os.mkdir(path)

class kiwoomApi():
    def __init__(self):
        self.ocx = QAxWidget('KHOPENAPI.KHOpenAPICtrl.1')
        self.ocx.OnEventConnect.connect(self.LoginEvent)
        self.ocx.OnReceiveTrData.connect(self.trEvent)

    def CommConnect(self):
        self.ocx.dynamicCall('CommConnect()')
        self.login_event_loop = QEventLoop()
        self.login_event_loop.exec_()
        
    def LoginEvent(self, err_code):
        if err_code == 0:
            print('Login 성공')
        else :
            print('Login 실패')
        self.login_event_loop.exit()
    
    def SetInputValue(self, sID, sValue):
        self.ocx.dynamicCall("SetInputValue(QString, QString)", sID, sValue)

    def CommRqData(self, sRQName, sTrCode, nPrevNext, BSTR):
        self.ocx.dynamicCall("CommRqData(QString, QString, QString, QString)", sRQName, sTrCode, nPrevNext, BSTR)
        # 이벤트 발생시키는 메서드에는 항상 EventLoop Exec 구현
        self.TrEventLoop = QEventLoop()
        self.TrEventLoop.exec_()
        
    def GetCommData(self, strTrCode, strRecordName, nIndex, strItemName):
        return self.ocx.dynamicCall("GetCommData(QString, QString, int, QString)", strTrCode, strRecordName, nIndex, strItemName).strip()

    def GetRepeatCnt(self, sTrCode, sRQName):
        return self.ocx.dynamicCall("GetRepeatCnt(QString, QString)", sTrCode, sRQName);
    
    def trEvent(self, sScrNo, sRQName, sTrCode, sRecordName, sPreNext, nDataLength, sErrorCode, sMessage, sSplmMsg):
            
        if sRQName == 'AT_opt10080':
            self.AT_opt10080(sTrCode, sRQName)
        # 이벤트 처리에는 항상 EventLoop Exit( )구현
        self.TrEventLoop.exit()
    
    def AT_opt10080(self, sTrCode, sRQName):
        
        dataCnt = self.GetRepeatCnt(sTrCode, sRQName);
        print('가져온 개수 : ', dataCnt)
        self.opt10080_data = {'체결시간': [], '거래량': [], '현재가':[], '시가':[], '고가':[], '저가':[]}
        
        for i in range(dataCnt):
            date2 = self.GetCommData(sTrCode, sRQName, i, "체결시간").strip()
            mount = self.GetCommData(sTrCode, sRQName, i, "거래량").strip()
            current = self.GetCommData(sTrCode, sRQName, i, "현재가").strip()
            start = self.GetCommData(sTrCode, sRQName, i, "시가").strip()
            high = self.GetCommData(sTrCode, sRQName, i, "고가").strip()
            low = self.GetCommData(sTrCode, sRQName, i, "저가").strip()
            
            self.opt10080_data['체결시간'].append(date2)
            self.opt10080_data['거래량'].append(mount)
            self.opt10080_data['현재가'].append(current)
            self.opt10080_data['시가'].append(start)
            self.opt10080_data['고가'].append(high)
            self.opt10080_data['저가'].append(low)
            print(date2, mount, current, start, high, low)
    
    # request값 1000건 이상일 때 time.sleep(3.6)
    # request값 1000건 이하일 때 time.sleep(0.6)
# 분봉데이터 822종목 900개 레코드, 2~3일치 -> 13분
# csv로 저장 시 약 40분
if __name__ == '__main__':
    
    app = QApplication(sys.argv) #QApplication 클래스의 인스턴스 생성
    api = kiwoomApi()
    api.CommConnect()
    
    stock_data_list = []
    
    for i, code in enumerate(kospi_df.code.iloc[:2]):
        
        print('code: %s, %i / %i 번째 코드' %(code,i+1,len(kospi_df.code.iloc[:2])))
        print('분봉체결시간','분봉거래량','분봉현재가','분봉시가','분봉고가','분봉저가')
        api.SetInputValue("종목코드", code)
        api.SetInputValue("틱범위", "1")
        api.SetInputValue("수정주가구분", "1")
        api.CommRqData("AT_opt10080", "opt10080", "0", "0102")
        con_datetime = api.opt10080_data['체결시간'] = pd.to_datetime(api.opt10080_data['체결시간'],\
                                                                  format="%Y%m%d%H%M%S")
        api.opt10080_data['체결시간'] = list(pd.Series(con_datetime)\
                                         .apply(lambda x: datetime.datetime.strftime(x, format='%Y-%m-%d %H:%M:%S')))
        stock_data = (code, api.opt10080_data)
        stock_data_list.append(stock_data)
        
        df = pd.DataFrame(stock_data_list[i][1])
        df1 = pd.merge(base_df, df, how='left', on='체결시간')
        df1.to_csv(path+code+'.csv', encoding='utf-8-sig', index=False)
        path = path.replace('\\','/')
        
        sql= """
        LOAD DATA LOCAL INFILE '{}{}.csv'
        INTO TABLE a{}
        FIELDS TERMINATED BY ','
        ENCLOSED BY '"'
        LINES TERMINATED BY '\n'
        IGNORE 1 LINES
        """.format(path,code,code)
        cursor.execute(sql)
        db.commit()
        
        time.sleep(0.6)

Login 성공
code: 095570, 1 / 2 번째 코드
분봉체결시간 분봉거래량 분봉현재가 분봉시가 분봉고가 분봉저가
가져온 개수 :  900
20211213153000 49381 -5270 -5270 -5270 -5270
20211213151900 3726 -5270 -5240 -5270 -5240
20211213151800 23 -5250 -5250 -5250 -5240
20211213151700 101 -5250 -5250 -5250 -5250
20211213151600 22 -5260 -5260 -5260 -5260
20211213151300 15 -5260 -5250 -5260 -5250
20211213151200 49 -5250 -5240 -5250 -5240
20211213151100 934 -5250 -5250 -5250 -5240
20211213150800 1 -5260 -5260 -5260 -5260
20211213150500 1 -5260 -5260 -5260 -5260
20211213150300 10 -5260 -5260 -5260 -5260
20211213150200 15 -5260 -5250 -5260 -5250
20211213150100 11 -5260 -5260 -5260 -5260
20211213150000 486 -5250 -5230 -5260 -5230
20211213145900 24 -5260 -5260 -5260 -5250
20211213145800 558 -5260 -5240 -5260 -5240
20211213145700 81 -5240 -5260 -5260 -5230
20211213145600 44 -5260 -5250 -5260 -5250
20211213145500 36 -5260 -5240 -5260 -5240
20211213145400 438 -5230 -5230 -5230 -5230
20211213145300 41 -5230 -5250 -5250 -5230
20211213145200 25 -5250 -52

20211210122900 51 -5310 -5300 -5310 -5300
20211210122800 54 -5300 -5290 -5300 -5290
20211210122700 49 -5290 -5290 -5290 -5290
20211210122600 25 -5290 -5300 -5300 -5290
20211210122400 64 -5310 -5300 -5310 -5290
20211210122300 41 -5290 -5290 -5300 -5290
20211210122100 1335 -5310 -5300 -5310 -5300
20211210122000 25 -5300 -5300 -5300 -5300
20211210121900 30 -5300 -5300 -5300 -5300
20211210121800 50 -5300 -5300 -5300 -5300
20211210121700 24 -5300 -5300 -5300 -5300
20211210121600 25 -5300 -5300 -5300 -5300
20211210121500 55 -5300 -5300 -5300 -5300
20211210121400 24 -5300 -5300 -5300 -5300
20211210121300 24 -5300 -5300 -5300 -5300
20211210121200 323 -5300 -5300 -5300 -5300
20211210121100 35 -5300 -5310 -5310 -5300
20211210121000 30 -5300 -5300 -5300 -5300
20211210120900 49 -5300 -5300 -5300 -5300
20211210120800 25 -5300 -5300 -5300 -5300
20211210120700 30 -5290 -5290 -5290 -5290
20211210120600 1 -5300 -5300 -5300 -5300
20211210120500 74 -5290 -5290 -5290 -5290
20211210120400 26 -5300 -5290 -5

code: 006840, 2 / 2 번째 코드
분봉체결시간 분봉거래량 분봉현재가 분봉시가 분봉고가 분봉저가
가져온 개수 :  900
20211213153000 330 -20350 -20350 -20350 -20350
20211213151900 372 -20350 -20350 -20350 -20350
20211213151800 4000 -20400 -20350 -20400 -20350
20211213151700 2 -20300 -20300 -20300 -20300
20211213151500 301 -20300 -20350 -20350 -20300
20211213151300 1 -20300 -20300 -20300 -20300
20211213151100 101 -20300 -20300 -20300 -20300
20211213151000 50 -20300 -20300 -20300 -20300
20211213150800 5 -20300 -20300 -20300 -20300
20211213150600 1 -20300 -20300 -20300 -20300
20211213150500 5 -20350 -20350 -20350 -20350
20211213150400 25 -20350 -20350 -20350 -20350
20211213150300 1 -20300 -20300 -20300 -20300
20211213150100 102 -20300 -20300 -20300 -20300
20211213150000 5 -20300 -20300 -20300 -20300
20211213145900 1372 -20300 -20300 -20300 -20300
20211213145700 1 -20300 -20300 -20300 -20300
20211213145600 1 -20350 -20350 -20350 -20350
20211213145000 1 -20300 -20300 -20300 -20300
20211213144900 5 -20350 -20350 -20350 -20350
20211213

20211209122000 100 +20800 +20800 +20800 +20800
20211209121900 15 +20750 +20750 +20750 +20750
20211209121800 355 +20800 +20800 +20800 +20800
20211209121700 4 +20800 +20800 +20800 +20800
20211209121500 15 +20800 +20800 +20800 +20800
20211209121400 4 +20850 +20850 +20850 +20850
20211209121200 16 +20800 +20800 +20800 +20800
20211209121000 1 +20800 +20800 +20800 +20800
20211209120900 15 +20750 +20750 +20750 +20750
20211209120800 18 +20750 +20750 +20750 +20750
20211209120500 15 +20750 +20750 +20750 +20750
20211209120200 15 +20750 +20750 +20750 +20750
20211209115800 1489 +20750 +20800 +20900 +20750
20211209115500 15 +20700 +20700 +20700 +20700
20211209115100 15 +20700 +20700 +20700 +20700
20211209114900 147 +20750 +20750 +20750 +20700
20211209114800 31 +20750 +20750 +20750 +20750
20211209114500 16 +20750 +20750 +20750 +20750
20211209114100 15 +20750 +20750 +20750 +20750
20211209113800 15 +20750 +20750 +20750 +20750
20211209113400 5 +20800 +20800 +20800 +20800
20211209113100 16 +20750 +20750 +

Wall time: 7.12 s


In [ ]:
sql= """
LOAD DATA LOCAL INFILE '{}{}.csv'
INTO TABLE a{}
FIELDS TERMINATED BY ','
ENCLOSED BY '"'
LINES TERMINATED BY '\n'
IGNORE 1 LINES
""".format(path,code,code)

In [2]:
pd.read_sql('select * from a006840',db)

,분봉체결시간,분봉거래량,분봉현재가,분봉시가,분봉고가,분봉저가
0,2021-12-13 15:30:00,330,-20350,-20350,-20350,-20350
1,2021-12-13 15:29:00,0,0,0,0,0
2,2021-12-13 15:28:00,0,0,0,0,0
3,2021-12-13 15:27:00,0,0,0,0,0
4,2021-12-13 15:26:00,0,0,0,0,0
...,...,...,...,...,...,...
386,2021-12-13 09:04:00,4,-20700,-20700,-20700,-20700
387,2021-12-13 09:03:00,86,-20700,20800,20800,-20650
388,2021-12-13 09:02:00,0,0,0,0,0
389,2021-12-13 09:01:00,327,-20700,20800,20800,-20700


In [ ]:
cursor.execute('truncate a095570')
db.commit()